In [2]:
from interpret.blackbox import LimeTabular
from interpret import show
import json
from tensorflow.keras.models import model_from_json

In [20]:
import numpy as np
import lime
import lime.lime_tabular

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler

In [1]:
import sys
sys.path.append("../")
import pandas as pd
from utils.data_utils import DataLoader

In [6]:
data_loader = DataLoader(file_path='../data/raw/Churn_Modelling.xls' , target_column='Exited', test_size=0.2, random_state=42, clean_data=True)
df = data_loader.encoded_data
df

,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1.0,0.0,0.0,1.0,0.0,619,42,2,0.00,1,1,1,101348.88
1,0.0,0.0,1.0,1.0,0.0,608,41,1,83807.86,1,0,1,112542.58
2,1.0,0.0,0.0,1.0,0.0,502,42,8,159660.80,3,1,0,113931.57
3,1.0,0.0,0.0,1.0,0.0,699,39,1,0.00,2,0,0,93826.63
4,0.0,0.0,1.0,1.0,0.0,850,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,0.0,1.0,771,39,5,0.00,2,1,0,96270.64
9996,1.0,0.0,0.0,0.0,1.0,516,35,10,57369.61,1,1,1,101699.77
9997,1.0,0.0,0.0,1.0,0.0,709,36,7,0.00,1,0,1,42085.58
9998,0.0,1.0,0.0,0.0,1.0,772,42,3,75075.31,2,1,0,92888.52


In [10]:
X_train, y_train = data_loader.get_train_data_enc()
X_test, y_test = data_loader.get_test_data_enc()

In [11]:
scale_columns = ['Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'Tenure']
scaler = MinMaxScaler()
X_train[scale_columns] = scaler.fit_transform(X_train[scale_columns])
X_test[scale_columns] = scaler.fit_transform(X_test[scale_columns])

In [12]:
model_architecture_path = r'C:\Users\ayesha.amjad\Documents\GitHub\MSDS-ML-Project\ML1-XAI\Project\models\NeuralNet\architecture.json'
model_weights_path = r'C:\Users\ayesha.amjad\Documents\GitHub\MSDS-ML-Project\ML1-XAI\Project\models\NeuralNet\weights.h5'

with open(model_architecture_path, 'r') as f:
    model_json = f.read()

In [15]:
model = model_from_json(model_json)
model.load_weights(model_weights_path)
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
#print("Classification Report: \n", classification_report(y_test, y_pred))

62/62 [==============================] - 0s 1ms/step


In [45]:
def predict_fn(data):
 # Replace with your preprocessing steps

    # Pass preprocessed_data through your neural network model and return predictions
    predictions = model.predict_proba(X_test)
    return predictions

In [39]:
X_train.columns.tolist()

['Geography_France',
 'Geography_Germany',
 'Geography_Spain',
 'Gender_Female',
 'Gender_Male',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [40]:
df.columns.tolist()

['Geography_France',
 'Geography_Germany',
 'Geography_Spain',
 'Gender_Female',
 'Gender_Male',
 'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [41]:
#Create a LIME explainer
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(df),  # Your training data
    feature_names=X_train.columns.tolist(),  # Names of your features
    class_names=['Existed'],  # Names of your target classes (optional)
    discretize_continuous=True,
    mode='classification'  # 'classification' or 'regression'
)



In [46]:
sample_idx = 0
# Generate explanations
explanations = explainer.explain_instance(
    X_test.iloc[sample_idx], predict_fn, num_features=5
)

# Interpret the explanations
explanations.show_in_notebook() 

AttributeError: 'Sequential' object has no attribute 'predict_proba'